# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
# with open("real_estate_sales_data.txt",encoding="utf-8") as f:
#     real_estate_sales = f.read()

In [1]:
with open("qa.txt",encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 有机食品是否真的更健康？\n[营养师回答] 有机食品可能含有较少的农药残留，但就营养价值而言，有机和非有机食品通常相似。')

In [6]:
len(docs)

49

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [8]:
!pip install faiss-cpu

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [23]:
!pip install -U langchain-openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url = 'https://api.xiaoai.plus/v1', api_key = 'sk-BBdn99z7cZuhVylF6fE5BdAf7735479d9bE4854f8a161067'))

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "如何控制体重"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 吃零食会导致体重增加吗？
[营养师回答] 适量、健康的零食不会导致体重增加，关键在于控制总热量摄入。

[客户问题] 吃太多碳水化合物会胖吗？
[营养师回答] 过量摄入碳水化合物，尤其是精制碳水化合物，可能导致体重增加。

[客户问题] 吃太多脂肪会胖吗？
[营养师回答] 过量摄入任何类型的食物都可能导致体重增加，关键在于控制总热量摄入。

[客户问题] 吃素是否有助于减肥？
[营养师回答] 吃素可以帮助控制体重，但关键在于整体饮食的均衡和热量摄入。



In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023F72F551C0>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 吃零食会导致体重增加吗？
[营养师回答] 适量、健康的零食不会导致体重增加，关键在于控制总热量摄入。

[客户问题] 吃太多碳水化合物会胖吗？
[营养师回答] 过量摄入碳水化合物，尤其是精制碳水化合物，可能导致体重增加。

[客户问题] 吃太多脂肪会胖吗？
[营养师回答] 过量摄入任何类型的食物都可能导致体重增加，关键在于控制总热量摄入。



In [15]:
docs = topK_retriever.get_relevant_documents("有健康的作息时间提供吗？")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 早餐真的很重要吗？
[营养师回答] 早餐可以帮助启动你的新陈代谢，提供能量，是一天中重要的一餐。

[客户问题] 咖啡对健康有害吗？
[营养师回答] 适量饮用咖啡对大多数人是安全的，过量则可能带来负面影响。

[客户问题] 牛奶真的有助于睡眠吗？
[营养师回答] 牛奶含有色氨酸，可能有助于睡眠，但效果因人而异。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [20]:
docs[0].page_content

IndexError: list index out of range

In [21]:
docs[0].page_content.split("[营养师回答] ")

IndexError: list index out of range

In [22]:
ans = docs[0].page_content.split("[营养师回答] ")[-1]

IndexError: list index out of range

In [23]:
ans

Document(page_content='[客户问题] 吃素是否有助于减肥？\n[营养师回答] 吃素可以帮助控制体重，但关键在于整体饮食的均衡和热量摄入。')

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "我想减肥"

print(sales(query))

[]


D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [26]:
print(sales(query, 0.75))

[]


D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


In [27]:
query = "成年人每天需要摄入的维生素为多少"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['[客户问题] 每天需要喝八杯水吗？\n[营养师回答] 每日饮水量应根据个人体重、活动量和气候调整，并非固定八杯。']

score:0.5 ans: ['[客户问题] 每天需要喝八杯水吗？\n[营养师回答] 每日饮水量应根据个人体重、活动量和气候调整，并非固定八杯。', '[客户问题] 吃太多维生素补充剂有害吗？\n[营养师回答] 过量摄入某些维生素可能有害，建议通过均衡饮食获取所需营养。', '[客户问题] 吃太多维生素A会对身体有害吗？\n[营养师回答] 过量摄入维生素A可能导致中毒，应按推荐量摄入。', '[客户问题] 吃太多B族维生素会对身体有害吗？\n[营养师回答] B族维生素大多为水溶性，过量会通过尿液排出，但某些B族维生素过量仍可能有害。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5, base_url = 'https://api.xiaoai.plus/v1', api_key = 'sk-BBdn99z7cZuhVylF6fE5BdAf7735479d9bE4854f8a161067')
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [30]:
qa_chain({"query": "牛油果搭配什么吃更好"})

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '牛油果搭配什么吃更好',
 'result': '牛油果搭配的食物很多样，可以根据个人口味选择。常见的搭配有：\n\n1. 面包：将牛油果捣碎后涂在烤面包上，可以加一些盐、胡椒或柠檬汁增加风味。\n2. 沙拉：牛油果切块加入各种蔬菜沙拉中，如番茄、黄瓜、生菜等，再配上橄榄油和醋。\n3. 鸡蛋：牛油果与煎蛋或煮蛋一起食用，营养丰富。\n4. 寿司：牛油果是很多寿司里不可缺少的成分，如加州卷。\n5. 墨西哥菜：牛油果是墨西哥菜中的重要食材，可用来制作鳄梨莎莎酱或塔可餐点。\n\n这些只是一些基本的搭配方式，牛油果的用途非常广泛，可以根据个人喜好进行创新搭配。'}

In [31]:
qa_chain({"query": "怎么能睡好"})

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '怎么能睡好',
 'result': '要想睡好觉，可以尝试以下一些方法：\n\n1. 保持规律的睡眠时间：每天尽量在同一时间上床睡觉和起床。\n2. 创造良好的睡眠环境：确保你的卧室安静、黑暗和适当的温度。\n3. 避免晚上摄入咖啡因和酒精：它们可能会干扰你的睡眠。\n4. 减少晚上使用电子设备的时间：蓝光可能会影响你的睡眠质量。\n5. 增加白天的身体活动：适度的运动可以帮助改善睡眠质量。\n\n如果你尝试了这些方法但仍然有睡眠问题，建议咨询医生或睡眠专家。'}

In [32]:
print(sales("喝牛奶会长痘吗"))

[]


D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [36]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(base_url = 'https://api.xiaoai.plus/v1', api_key = 'sk-BBdn99z7cZuhVylF6fE5BdAf7735479d9bE4854f8a161067'),allow_dangerous_deserialization = True)

In [38]:
!pip install -U langchain-openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/46.7 kB ? eta -:--:--
     --------------------------------- ---- 41.0/46.7 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 46.7/46.7 kB 776.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/373.5 kB ? eta -:--:--
     ----------- -------------------------- 112.6/373.5 kB 6.8 MB/s eta 0:00:01
     -------------------------- ----------- 256.0/373.5 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 373.5/373.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.19
    Uninstalling langchain-core-0.2.19:
      Successfully uninstalled langchain-core-0.2.19
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.1.16
    Uninstalling langchain-openai-0.1.16:
      Successfully uninstalled langchain-openai-0.1.16


In [39]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5,base_url = 'https://api.xiaoai.plus/v1', api_key = 'sk-BBdn99z7cZuhVylF6fE5BdAf7735479d9bE4854f8a161067')
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [40]:
qa_chain({"query": "我想减肥"})

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想减肥',
 'result': '那你可以考虑改变饮食习惯和增加运动。首先，你需要保证你的饮食健康均衡，包括大量的水果和蔬菜，少吃糖分和油脂高的食物。其次，定期做运动也很重要，比如散步，跑步，游泳或者做一些力量训练。同时，保持足够的睡眠和积极的心态也对减肥有帮助。如果需要，你也可以寻求专业营养师或者健身教练的帮助。'}

In [41]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [42]:
qa_chain({"query": "我想减肥"})

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想减肥',
 'result': '你可以尝试健康饮食和规律的锻炼来达到减肥的目标。避免过度摄入糖分和脂肪，多吃蔬菜，水果和全谷类。每天至少做30分钟的有氧运动，比如步行，跑步或者游泳。如果可能的话，你也可以找一个营养师或者健身教练来帮助你制定一个适合你的减肥计划。记住，减肥不是一蹴而就的，需要时间和耐心。'}

In [43]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [44]:
result = qa_chain({"query": "我想减肥"})

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [45]:
result

{'query': '我想减肥',
 'result': '您可以尝试以下的方法来减肥：\n1. 健康饮食：尽量吃全谷类，蔬菜，水果，瘦肉和低脂乳制品，并且控制热量摄入。\n2. 常规运动：每周至少做150分钟的中等强度运动，或者75分钟的高强度运动。\n3. 喝水：喝足够的水可以帮助您控制食欲并保持身体健康。\n4. 确保足够的睡眠：缺乏睡眠可能会导致体重增加。\n5. 避免压力：压力和焦虑可能会导致过度进食。\n如果你的体重问题很严重，你可能需要寻求医生或营养师的帮助。',
 'source_documents': []}